# Testing CBOR implementations for map encoding canonicality

Tested:
- [Julia](https://github.com/JuliaIO/CBOR.jl)
- [Rust](https://docs.rs/serde_cbor/0.11.0/serde_cbor/)
- [JS cbor](https://www.npmjs.com/package/cbor)
- [JS borc](https://www.npmjs.com/package/borc)

In [227]:
using CBOR, JSON, Test

In [217]:
data_dict = Dict("a" => 1, "b" => 2, "c" => Dict("b" => 1, "a" => 2))
data_json = json(data_dict)

"{\"c\":{\"b\":1,\"a\":2},\"b\":2,\"a\":1}"

In [218]:
data_json = """{"iss":"BBmEKZciGMonT_G0CmiM4HdfM6o0ktuh3xIFadvc1TVgA0ZJUNIS6go0pX8jwSUorbDfv27T_M9M9wldMFk6t00","iat":1578835560,"sub":"geo:?q=47.1691576,8.514572(Juanitos)&u=30","rating":75,"metadata":{"display_name":"dsafsa","given_name":"sadf","age":23,"gender":"Female","client_uri":"https://mangrove.reviews"}}"""
data_dict = JSON.parse(data_json)

Dict{String,Any} with 5 entries:
  "sub"      => "geo:?q=47.1691576,8.514572(Juanitos)&u=30"
  "iat"      => 1578835560
  "iss"      => "BBmEKZciGMonT_G0CmiM4HdfM6o0ktuh3xIFadvc1TVgA0ZJUNIS6go0pX8jwS…
  "rating"   => 75
  "metadata" => Dict{String,Any}("client_uri"=>"https://mangrove.reviews","gend…

In [219]:
encoded_jl = encode(data_dict)
length(encoded_jl)

280

In [220]:
encoded_js_borc = `node js/cbor.js borc encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_js_borc)

261

In [221]:
encoded_js_cbor = `node js/cbor.js cbor encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_js_cbor)

261

In [222]:
encoded_rs = `./rust/target/debug/rust encode $data_json` |> open |> JSON.parse |> Vector{UInt8}
length(encoded_rs)

261

In [230]:
@testset "JS CBOR libs" begin
  @test encoded_js_borc == encoded_js_cbor
  @test decode(encoded_js_borc) == decode(encoded_js_cbor)
end

JS CBOR libs: Test Failed at In[230]:2
  Expression: encoded_js_borc == encoded_js_cbor
   Evaluated: UInt8[0xa5, 0x63, 0x69, 0x61, 0x74, 0x1a, 0x5e, 0x1b, 0x1e, 0x68  …  0x61, 0x6d, 0x65, 0x66, 0x64, 0x73, 0x61, 0x66, 0x73, 0x61] == UInt8[0xa5, 0x63, 0x69, 0x73, 0x73, 0x78, 0x57, 0x42, 0x42, 0x6d  …  0x76, 0x65, 0x2e, 0x72, 0x65, 0x76, 0x69, 0x65, 0x77, 0x73]
Stacktrace:
 [1] top-level scope at In[230]:2
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Test/src/Test.jl:1107
 [3] top-level scope at In[230]:2
Test Summary: | Pass  Fail  Total
JS CBOR libs  |    1     1      2


TestSetException: Some tests did not pass: 1 passed, 1 failed, 0 errored, 0 broken.

In [231]:
@testset "JS vs Rust CBOR libs" begin
  @test encoded_js_borc == encoded_rs
  @test decode(encoded_js_borc) == decode(encoded_rs)
end

Test Summary:        | Pass  Total
JS vs Rust CBOR libs |    2      2


Test.DefaultTestSet("JS vs Rust CBOR libs", Any[], 2, false)

In [232]:
@testset "Julia vs Rust CBOR libs" begin
  @test encoded_rs == encoded_jl
  @test decode(encoded_rs) == decode(encoded_jl)
end

Julia vs Rust CBOR libs: Test Failed at In[232]:2
  Expression: encoded_rs == encoded_jl
   Evaluated: UInt8[0xa5, 0x63, 0x69, 0x61, 0x74, 0x1a, 0x5e, 0x1b, 0x1e, 0x68  …  0x61, 0x6d, 0x65, 0x66, 0x64, 0x73, 0x61, 0x66, 0x73, 0x61] == UInt8[0xa5, 0x63, 0x73, 0x75, 0x62, 0x78, 0x29, 0x67, 0x65, 0x6f  …  0x65, 0x3b, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xe8]
Stacktrace:
 [1] top-level scope at In[232]:2
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Test/src/Test.jl:1107
 [3] top-level scope at In[232]:2
Test Summary:           | Pass  Fail  Total
Julia vs Rust CBOR libs |    1     1      2


TestSetException: Some tests did not pass: 1 passed, 1 failed, 0 errored, 0 broken.

## Results

All libraries manage to preserve semantics of encoded message.

- Julia uses some tags during encoding resulting in a different length message.
- JS `cbor` library does not seem to encode canonical CBOR.
- Rust encodes canonical CBOR as long as it does not do it from a `Serialize`able struct, but from `serde_cbor::Value` 